# ground truth 加入 Qdrant 並預測

In [94]:
import pandas as pd
from ast import literal_eval
import qdrant_client # Qdrant's client library for Python
from sentence_transformers import SentenceTransformer
import re
from qdrant_client.http import models as rest
from qdrant_client.models import PointStruct # Import the PointStruct to store the vector and payload
from tqdm import tqdm # Library to show the progress bar 

# emoji範圍定義
emojis = '[\U0001F170-\U0001F251\U0001F600-\U0001F64F\U0001F300-\U0001F6FF\U0001F900-\U0001FAFF\U0001F680-\U0001F77F\U00002600-\U000026FF\U0001F354-\U0001F37F\u2B00-\u2BFF\uFE0F\U0001F7E0-\U0001F7EB\U0001F1E6-\U0001F1FF\u2190-\u21FF\u2B00-\u2BFF\u27A0-\u27BF\u2934\u2935]'

def remove_text_between_same_emoji(text):
    '''移除兩個相同emoji之間的所有字'''
    global emojis
    pattern = re.compile(rf"({emojis})[^\\1]*\1")
    while re.search(pattern, text):
        text = re.sub(pattern, "", text)  # 反覆處理所有匹配的情況
    return text

def fullwidth_to_halfwidth(text):
    '''全形到半形的對應字符直接對照轉換'''
    fullwidth = "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ！＂＃＄％＆＇（）＊＋，－．／：；＜＝＞？＠［＼］＾＿｀｛｜｝～"
    halfwidth = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    
    # 使用 str.maketrans() 創建映射表並應用於文本
    return text.translate(str.maketrans(fullwidth, halfwidth))

def clean_text(text, tokenization=True):
    global emojis
    
    text = fullwidth_to_halfwidth(text)
    text = remove_text_between_same_emoji(text)
    text = re.sub(emojis, "", text)
    text = re.sub(r'[【】\[\]「」()（）〔〕《》『』]', '', text)
    text = re.sub(r'\s*([*×X]|元|支|粒|公克|克|毫克|毫升|公升|公斤|公釐|公分|尺|公尺|吋|寸|片|碗|袋|盒|包|瓶|罐|入|組|盒|冊|l|L|mg|ml|kg|ML|Kg|KG|cc|CC|g|G|oz|mm|cm|CM)\s*', r'\1', text)# 刪除乘號和單位周圍的空白
    text = re.sub(r'\d+(\.\d+)?(元|支|粒|公克|克|毫克|毫升|公升|公斤|公釐|公分|尺|公尺|吋|寸|片|碗|袋|盒|包|瓶|罐|入|組|盒|冊|l|L|mg|ml|kg|ML|Kg|KG|cc|CC|g|G|oz|mm|cm|CM)([*×X]?\d+(\.\d+)?(元|支|粒|公克|克|毫克|毫升|公升|公斤|公釐|公分|尺|公尺|吋|寸|片|碗|袋|盒|包|瓶|罐|入|組|盒|冊|l|L|mg|ml|kg|ML|Kg|KG|cc|CC|g|G|oz|mm|cm|CM)?)?', '', text)

    delete_words = [
        "特價", "出貨", "單賣", "台灣", "日本", "韓國", "德國", "美國", "瑞士", "日韓", "直送", "免運", "宅配", 
        "單入", "快速出貨", "多一件", "限時", "限量", "現貨", "缺貨", "代銷", "團購", "加購", "加價", "期間限定",
        "加價購", "預購", "原價", "滿千折百", "新春", "優惠", "年終回饋", "新品上市", "獨享", "下殺", "快閃"
    ]

    # Go 定義
    delete_words = delete_words + ["折扣", "折抵", "贈送", "滿額", "扣抵", "即享券", "金流", "省", "隨機",
                                    "推薦", "單瓶", "任選", "贈", "抵用券", "換", "換購", "餐點", "電子憑證" 
                                   "兌換券", "回訪", "環保杯", "杯扣", "免費", "晚餐", "午餐", "早餐", "體驗",
                                    "組合", "效期", "試用", "試用包", "限定", "兌禮", "小包裝", "生日", "隨餐提供", 
                                    "隨餐上", "贈品", "件贈", "訂單", "訂金", "消費", "即贈", "單點", "員工", "成人",
                                    "無糖", "去冰", "少冰", "一分糖", "三分糖", "少冰", "後上", "先上", "小辣",
                                    "中辣", "大辣", "款式", "單筆", "不限", "金額", "全館", "抽獎券", "不累", "回購",
                                    "送完為止", "禮包", "金幣", "外帶", "內用", "感謝", "享受", "配送", "促", "促銷",
                                    "購物袋", "好康", "方案", "好康", "好康", "正價品", "套餐", "好禮", "使用", "錢包"]

    # # 將所有刪除詞語合併成一個正則表達式（使用 | 作為"或"的意思）
    delete_pattern = '|'.join([re.escape(word) for word in delete_words])

    # # 使用正則表達式一次性刪除所有這些詞語
    text = re.sub(rf'({delete_pattern})', '', text)

        
    # # 刪除「買2送1」和「7折」格式
    text = re.sub(r'買[1-9]送[1-9]', '', text)  # 刪除 "買X送X" 格式
    text = re.sub(r'[1-9]折', '', text)  # 刪除 "X折" 格式

    # # 刪除「買一送一」和「三折」格式
    text = re.sub(r'買[一二三四五六七八九]送[一二三四五六七八九]', '', text)  # 刪除 "買X送X" 格式
    text = re.sub(r'[一二三四五六七八九]折', '', text)  # 刪除 "X折" 格式


    text = re.sub(r'[㊣�?™±、°:;~!@#$%^&*_\-+※,.，。\\/]', '', text)
    text = re.sub(r"[\"']", '', text)
    text = re.sub(r'[A-Za-z]*[0-9][A-Za-z0-9/]*', '', text)
    text = re.sub(r'\s+', ' ', text)  # 合併多餘的空白
    text = re.sub(r'\d{3,}', '', text)# 去除3個以上的數字
    text = text.strip()

    # text = re.sub(r"^[A-Za-z\d]+", "", text)
    # text = re.sub(r"\(.*?\)|\[.*?\]|\【.*?\】|\（.*?\）|\《.*?\》", "", text) # => (取捨)刪除太多資訊
    # text = re.sub(r"[\d.]+[a-zA-Z]*[xX*／/]?\d*[粒克片碗袋公斤盒包瓶罐入組mlkgKGML].*", "", text)
    # text = re.sub(r'\s*([*×X元粒公克克公升公斤公釐公分尺公尺吋寸片碗袋盒包瓶罐入組lLmlkgMLKgKGccCCgGozmmcmCM])\s*', r'\1', text)# 刪除乘號和單位周圍的空白
    # text = re.sub(r'\d+(\.\d+)?( ㄕ元|粒|公克|克|公升|公斤|公釐|公分|尺|公尺|吋|寸|片|碗|袋|盒|包|瓶|罐|入|組|盒|l|L|ml|kg|ML|Kg|KG|cc|CC|g|G|oz|mm|cm|CM)([*×X]?\d+(\.\d+)?(元|粒|公克|克|公升|公斤|公釐|公分|尺|公尺|吋|寸|片|碗|袋|盒|包|瓶|罐|入|組|盒|l|L|ml|kg|ML|Kg|KG|cc|CC|g|G|oz|mm|cm|CM)?)?', '', text)
    # text = re.sub(r"[／/]{1}[粒克片碗袋公斤盒包瓶罐入組mlkgKGML].*", "", text)
    # text = re.sub(r"[*＊\-_－xX%]+", "", text)
    # text = re.sub(r"(PET){1}\d*", "", text)

    # if "平台費" in text or "外送費" in text:
    #     text = text.split(" ")[0]

    # if tokenization:
    #     text = " ".join(jieba.cut(text))

    return text

In [95]:
# 使用 SentenceTransformer 生成文本嵌入 
# model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
model = SentenceTransformer('intfloat/multilingual-e5-base')

In [96]:
# 載入數據
df = pd.read_csv('/Users/gohuang/Documents/OneAD/invoice_process/classifier_model_doc/groundtruth/0418_GT綜合版.csv')
print(df)

# Embedding 資料
products = df['product'].tolist()
emb_products = model.encode(products, batch_size=32, show_progress_bar=True)
df['emb_products'] = emb_products.tolist()
print(df)

        product  category_id category  sub_category product_industry
0         橘色一零一          103     文具用品  教具/文具用品/辦公用品             生活家居
1         文具背心袋          103     文具用品  教具/文具用品/辦公用品             生活家居
2      修正內帶藍 手牌          103     文具用品  教具/文具用品/辦公用品             生活家居
3         綠色一零一          103     文具用品  教具/文具用品/辦公用品             生活家居
4        鉛筆芯 百點          103     文具用品  教具/文具用品/辦公用品             生活家居
...         ...          ...      ...           ...              ...
11563       購物袋           98  居家收納與清潔          收納用品             生活家居
11564    UBER運費          121     其他服務         物流/運費            交通與服務
11565     貼心紙袋大           98  居家收納與清潔          收納用品             生活家居
11566     貼心紙袋中           98  居家收納與清潔          收納用品             生活家居
11567     拋棄式隱眼          152  醫療器材與藥品          藥品護理             醫藥保健

[11568 rows x 5 columns]


Batches: 100%|██████████| 362/362 [00:31<00:00, 11.40it/s]


        product  category_id category  sub_category product_industry  \
0         橘色一零一          103     文具用品  教具/文具用品/辦公用品             生活家居   
1         文具背心袋          103     文具用品  教具/文具用品/辦公用品             生活家居   
2      修正內帶藍 手牌          103     文具用品  教具/文具用品/辦公用品             生活家居   
3         綠色一零一          103     文具用品  教具/文具用品/辦公用品             生活家居   
4        鉛筆芯 百點          103     文具用品  教具/文具用品/辦公用品             生活家居   
...         ...          ...      ...           ...              ...   
11563       購物袋           98  居家收納與清潔          收納用品             生活家居   
11564    UBER運費          121     其他服務         物流/運費            交通與服務   
11565     貼心紙袋大           98  居家收納與清潔          收納用品             生活家居   
11566     貼心紙袋中           98  居家收納與清潔          收納用品             生活家居   
11567     拋棄式隱眼          152  醫療器材與藥品          藥品護理             醫藥保健   

                                            emb_products  
0      [-0.015984948724508286, 0.02553623542189598, -...  
1      [0.0176701

In [97]:
df.info(show_counts=True)
# Get the vector size from the first row to set up the collection


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11568 entries, 0 to 11567
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product           11568 non-null  object
 1   category_id       11568 non-null  int64 
 2   category          11568 non-null  object
 3   sub_category      11568 non-null  object
 4   product_industry  11568 non-null  object
 5   emb_products      11568 non-null  object
dtypes: int64(1), object(5)
memory usage: 542.4+ KB


## 連線 Qdrant

In [99]:
qdrant = qdrant_client.QdrantClient(host="34.80.206.199", port=6333)
qdrant.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='group_with_cate'), CollectionDescription(name='classifier_0421'), CollectionDescription(name='product_category'), CollectionDescription(name='google_interest')])

## 建立 Qdrant Collection

In [98]:
# 確認目前的 size 是否跟 Embedding 相同
vector_size = len(df['emb_products'][0])
print(vector_size)

768


In [ ]:
qdrant.recreate_collection(
    collection_name='AKC1128',
    vectors_config={
        'product': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
        'classifier': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

/var/folders/y5/jf5bwk4s4t5c5c15cy6sq2zm0000gn/T/ipykernel_46415/3347798691.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

## 上傳資料 
- collection_name 記得要換 

In [ ]:

# Populate collection with vectors using tqdm to show progress
for k, v in tqdm(df.iterrows(), desc="Upserting products", total=len(df)):
    try:
        qdrant.upsert(
            collection_name='AKC1128',
            points=[
                PointStruct(
                    id=k,
                    vector={'product': v['emb_products']},
                    payload={
                        'classifier': v['product_industry']+'-'+v['category']+'-'+v['sub_category'],
                    }
                    
                )
            ]
        )
    except Exception as e:
        print(f"Failed to upsert row {k}: {v}")
        print(f"Exception: {e}")

Upserting products: 100%|██████████| 11568/11568 [05:04<00:00, 38.04it/s]


In [ ]:
# 確認上傳筆數
qdrant.count(collection_name='AKC1128')

CountResult(count=11568)

## 清理商品資訊並預測

In [103]:
def query_qdrant(query, collection_name, vector_name='product', top_k=1):

    # Creates embedding vector from user query
    embedded_query = model.encode(query, batch_size=32, show_progress_bar=False)

    query_results = qdrant.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k, 
        query_filter=None
    )

    
    return query_results

def predict(query_ori, top_k):
    
    query = clean_text(query_ori)
    # if len(query.split(' ')) > 0:
    #     query = query.split(' ')[0]
    print("原商品：",query_ori, "> Clean後:",query)
    query_results = query_qdrant(query, 'classifier_0422', 'product', top_k)
    for i, products in enumerate(query_results):
        print(f'{i + 1}. {products.payload["classifier"]}, (Score: {round(products.score, 3)})')
        # return products.payload["category"], products.payload["sub_category"], round(products.score, 2)
        # return products.payload["classifier"], round(products.score, 2)




In [106]:
predict("一般環保杯", 3)

原商品： 一般環保杯 > Clean後: 一般
1. 醫藥保健-醫療器材與藥品-藥品護理, (Score: 0.927)
2. 醫藥保健-醫療器材與藥品-藥品護理, (Score: 0.927)
3. 美妝個清-臉部保養-臉部乳液, (Score: 0.913)


In [36]:
error_list = ['Xemenry酸櫻桃10:1提取物300毫克軟糖，相當於3000毫克酸櫻桃🍒',
              '女短POLO衫M湖藍(千凡)',
              '大同陳年蔭油膏',
              '美舒律 蒸氣眼罩 玫瑰花香 5片裝',
              'PILOT 4色變芯筆管-珠光粉 PLHKC20-PKP',
              '【SAMPO 聲寶】五段式音波震動牙刷超值組-白',
              '韓式辣味鍋貼',
              '起司牛肉可頌',
              '餐-大麥克',
              '購物袋-大袋PPI',
              '高露潔全面防護',
              '3M舒適繃綜合30片',
              '12h處理🚀紫南宮過爐開光🙏黃銅聚寶盆鑰匙扣 十二生肖招財鑰匙扣 轉運手把招財掛件 轉錢鑰匙扣 男女',
              'Q哥 真5D滿版 保護貼 玻璃貼 適用 iPhone 16 15 14 13 12 11 Pro M',
              'RANSO 聯碩 43型FHD低藍光液晶顯示器(RF-43VA3)',
              '不要 紙杯'
              ]

ori_predict = ['生鮮冷凍-乳蛋豆製品-養樂多/優酪乳/優格',
              '飲料零食-茶飲/咖啡-咖啡',
              '飲料零食-茶飲/咖啡-紅茶/花茶/水果茶',
              '個人清潔-洗髮護髮-洗髮',
              '生鮮冷凍-肉品-火鍋肉片',
              '個人清潔-口腔沐浴-牙刷',
              '餐飲美食-西式料理-牛排',
              '餐飲美食-韓式料理-鍋物',
              '餐飲美食-中式料理-飯類/麵類/火鍋',
              '食品飲料-生鮮食材-蔬菜水果/農特產',
              '運動戶外-運動健身-運動護具',
              '美妝個清-身體與手足保養-私密保養品',
              '家用電器-廚房相關-快煮壺/熱水瓶/咖啡機',
              '3C科技-手機與電腦-手機',
              '家用電器-室內居家-風扇/捕蚊家電',
              '餐飲美食-日式料理-壽司/生魚片'
              ]

for i in range(0,len(error_list)):
    print("預測結果 New:",predict(error_list[i]), "> Old:",ori_predict[i] )
    




原商品： Xemenry酸櫻桃10:1提取物300毫克軟糖，相當於3000毫克酸櫻桃🍒 > Clean後: Xemenry酸櫻桃提取物軟糖相當於酸櫻桃
預測結果 New: ('食品飲料-零食與甜點-堅果與果乾類', 0.9) > Old: 生鮮冷凍-乳蛋豆製品-養樂多/優酪乳/優格
原商品： 女短POLO衫M湖藍(千凡) > Clean後: 女短POLO衫M湖藍千凡
預測結果 New: ('服飾鞋帽-男女服飾-上衣', 0.89) > Old: 飲料零食-茶飲/咖啡-咖啡
原商品： 大同陳年蔭油膏 > Clean後: 大同陳年蔭油膏
預測結果 New: ('食品飲料-主食與調味料-抹醬/沾醬', 0.89) > Old: 飲料零食-茶飲/咖啡-紅茶/花茶/水果茶
原商品： 美舒律 蒸氣眼罩 玫瑰花香 5片裝 > Clean後: 美舒律 蒸氣眼罩 玫瑰花香 裝
預測結果 New: ('美妝個清-臉部保養-面膜', 0.92) > Old: 個人清潔-洗髮護髮-洗髮
原商品： PILOT 4色變芯筆管-珠光粉 PLHKC20-PKP > Clean後: PILOT 色變芯筆管珠光粉
預測結果 New: ('生活家居-文具用品-教具/文具用品/辦公用品', 0.91) > Old: 生鮮冷凍-肉品-火鍋肉片
原商品： 【SAMPO 聲寶】五段式音波震動牙刷超值組-白 > Clean後: SAMPO 聲寶五段式音波震動牙刷超值組白
預測結果 New: ('美妝個清-口腔護理-牙刷', 0.93) > Old: 個人清潔-口腔沐浴-牙刷
原商品： 韓式辣味鍋貼 > Clean後: 韓式辣味鍋貼
預測結果 New: ('餐飲美食-副餐類-小吃/小點類', 1.0) > Old: 餐飲美食-西式料理-牛排
原商品： 起司牛肉可頌 > Clean後: 起司牛肉可頌
預測結果 New: ('食品飲料-零食與甜點-烘焙甜點', 0.96) > Old: 餐飲美食-韓式料理-鍋物
原商品： 餐-大麥克 > Clean後: 餐大麥克
預測結果 New: ('餐飲美食-主食類-漢堡/大亨堡', 0.91) > Old: 餐飲美食-中式料理-飯類/麵類/火鍋
原商品： 購物袋-大袋PPI > Clean後: 大袋PPI
預測結果 New: ('生活家居-居家用品-家居裝飾/生活用品', 0.9

原商品： 一般環保杯 > Clean後: 一般
1. 醫藥保健-醫療器材與藥品-藥品護理, (Score: 0.927)
2. 醫藥保健-醫療器材與藥品-藥品護理, (Score: 0.927)
3. 美妝個清-臉部保養-臉部乳液, (Score: 0.913)


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-base')
model.save("/Users/gohuang/Documents/OneAD/invoice_process/e5_model")